# Importing Libraries

In [1]:
import cv2
from matplotlib import pyplot as plt
import numpy as np
import mediapipe as mp
import time
import os

# Setting up mediapipe

In [2]:
# Mediapipe model
mp_holistic = mp.solutions.holistic

# Variable to draw points on the frame
mp_drawing = mp.solutions.drawing_utils

In [3]:
def mp_detection(image, model):
    
    # Converting opencv captured image in BGR 2 RGB for mediapipe.
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) 
    
    # Set image non-writable
    image.flags.writeable = False
    
    # make prediction using mediapipe model. Result store the 
    results = model.process(image)
    
    # Set image writable
    image.flags.writeable = True
    
    # Converting image to BGR format again for display.
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    return image, results 

In [4]:
def draw_landmarks(image, results):
    
    # Draw face connections
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION,
                             mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), # Style the landmark(dot)
                             mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)# Style the connections
                             )
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 
    
    
# results.face_landmarks stores the co-ordinates of face landmark co-ordinates.

# Testing mediapipe integration and landmarks in webcam

In [5]:
# cap = cv2.VideoCapture(0)

# # Set mediapipe model
# with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
#     while cap.isOpened():

#         # Read Frame
#         ret, frame = cap.read()

#         # Make detection using mediapipe model
#         image, results = mp_detection(frame, holistic) 
#         print(results)
        
#         # Draw landmarks stored in results on the frame.
#         draw_landmarks(image, results)

#         # Show it on screen and setting screen title.
#         cv2.imshow('OpenCV Feed', image)

#         # Shutting camera window down
#         if cv2.waitKey(10) & 0xFF == ord('q'):
#             break
#     cap.release()
#     cv2.destroyAllWindows()

In [6]:
# len(results.pose_landmarks.landmark)

In [7]:
# draw_landmarks(frame, results)

In [8]:
# plt.imshow(frame)

# Extracting Keypoints

In [9]:
def extract_keypoints(results):

    # If left hand not in frame, create array of same zise as results.left_hand_landmarks.landmark . hand landmark does not have
    # visibility so 21 landmarks dots multiplied by 3 co-ordinates. number of dots can be determined by bringing hand into frame 
    # and printing the array results.left_hand_landmarks.landmark. We need to create an empty array if no hand is present in 
    # frame so that all the features go into LSTM.
    lh = np.array([[res.x,res.y,res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)

    # Right hand landmarks
    rh = np.array([[res.x,res.y,res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)


    # (results.pose_landmarks.landmark) gives an array of all points for pose stored in results
    pose = np.array([[res.x,res.y,res.z,res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(132)

    # Face landmarks
    face = np.array([[res.x,res.y,res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(1404)
    
    return np.concatenate([pose, face, lh, rh])

In [10]:
# extract_keypoints(results).shape

# Collecting Traning data through webcam

### 1. Setup folders for data collection

In [11]:
# Path for the exported data, numpy arrays
DATA_PATH = os.path.join('MP_Data')

# Action to be detected
actions = np.array(['hello', 'thanks', 'iloveyou'])

# Thirty videos worth of data for each action.
no_sequence = 30

# Each video will be of 30 frames
sequence_length = 30

# Total data = (total no. of frames)*(no. of actions)*(Total keypoints) = 30*30*3*1662

for action in actions:
    for sequence in range(no_sequence):
        try:
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except:
            pass

### 2. Collecting Keypoint values for training and testing

In [12]:
# cap = cv2.VideoCapture(0)

# # Set mediapipe model
# with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
#     # for every action.
#     for action in actions:
#         # for every video for action 
#         for sequence in range(no_sequence):
#             # for every frame in every video
#             for frame_num in range(sequence_length):
                
#                 # Read Frame
#                 ret, frame = cap.read()

#                 # Make detection using mediapipe model
#                 image, results = mp_detection(frame, holistic) 
#                 print(results)

#                 # Draw landmarks stored in results on the frame.
#                 draw_landmarks(image, results)
                
#                 # Wait logic to get breaks between collecting data.
#                 if frame_num == 0:
#                     cv2.putText(image, 'Starting Collection',(120,200),
#                                 cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 4, cv2.LINE_AA)
#                     cv2.putText(image, 'Collecting frames for {} Video number {}'.format(action,sequence), (15,12),
#                                 cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
#                     cv2.waitKey(2000)
#                 else:
#                     cv2.putText(image, 'Collecting frames for {} Video number {}'.format(action,sequence), (15,12),
#                                 cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                
#                 # Exporting to our created folders
#                 keypoints = extract_keypoints(results)
#                 npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num)) 
#                 np.save(npy_path, keypoints)
                    
#                 # Show it on screen and setting screen title.
#                 cv2.imshow('OpenCV Feed', image)

#                 # Shutting camera window down
#                 if cv2.waitKey(10) & 0xFF == ord('q'):
#                     break
#     cap.release()
#     cv2.destroyAllWindows()

# Preprocessing data, creating labels and features. 

In [ ]:
# from sklearn.model_selection import train_test_split
# from tensorflow.keras.utils import to_categorical

In [ ]:
# label_map = {label:num for num, label in enumerate(actions)}

In [ ]:
# sequences, labels = [], []

# for action in actions:
#     for sequence in range(no_sequence):
#         video = []
#         for frame_num in range(sequence_length):
#             res = np.load(os.path.join(DATA_PATH, action, str(sequence), '{}.npy'.format(frame_num)))
#             video.append(res)
#         sequences.append(video)
#         labels.append(label_map[action])

In [ ]:
# X = np.array(sequences)
# y = to_categorical(labels).astype(int)

In [ ]:
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.05)

# Building and traning LSTM Neural Network

In [ ]:
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import LSTM, Dense
# from tensorflow.keras.callbacks import TensorBoard

In [ ]:
# log_dir = os.path.join('Logs')
# tb_callback = TensorBoard(log_dir = log_dir)

In [ ]:
# model = Sequential()
# model.add(LSTM(64, return_sequences = True, activation = 'relu', input_shape = (30,1662)))
# model.add(LSTM(128, return_sequences = True, activation = 'relu'))
# model.add(LSTM(64, return_sequences = False, activation = 'relu'))
# model.add(Dense(64, activation = 'relu'))
# model.add(Dense(32, activation = 'relu'))
# model.add(Dense(actions.shape[0], activation = 'softmax'))

In [ ]:
# model.compile(optimizer = 'Adam', loss = 'categorical_crossentropy', metrics = ['categorical_accuracy'])

In [ ]:
# model.fit(X_train, y_train, epochs = 500, callbacks = [tb_callback])

In [ ]:
# model.summary()

# Making Predictions

In [ ]:
# res = model.predict(X_test)

In [ ]:
# actions[np.argmax(res[4])]

In [ ]:
# actions[np.argmax(y_test[4])]

# Saving Model

In [ ]:
# model.save('sign_language_detection.h5')

# Evaluating using confusion matrix

In [ ]:
# from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

# ytrue = np.argmax(y_test, axis = 1).tolist()
# yhat = np.argmax(res, axis = 1).tolist()

In [ ]:
# multilabel_confusion_matrix(ytrue, yhat)

In [ ]:
# accuracy_score(ytrue, yhat)

# Test in real time

In [13]:
colors = [(245,117,16), (117,245,16), (16,117,245)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame

In [15]:
from tensorflow.keras.models import load_model
model = load_model('sign_language_detection.h5')

In [17]:
# 1. New detection variables
sequence = []
sentence = []
threshold = 0.8

cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mp_detection(frame, holistic)
        print(results)
        
        # Draw landmarks
        draw_landmarks(image, results)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-30:]
        
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])
            
            
        #3. Viz logic
            if res[np.argmax(res)] > threshold: 
                if len(sentence) > 0: 
                    if actions[np.argmax(res)] != sentence[-1]:
                        sentence.append(actions[np.argmax(res)])
                else:
                    sentence.append(actions[np.argmax(res)])

            if len(sentence) > 5: 
                sentence = sentence[-5:]

            # Viz probabilities
            image = prob_viz(res, actions, image, colors)
            
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

1/1 [==============================] - 0s 25ms/step
iloveyou
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 22ms/step
iloveyou
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 23ms/step
iloveyou
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 28ms/step
iloveyou
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 22ms/step
iloveyou
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 23ms/step
iloveyou
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 25ms/step
iloveyou
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 28ms/step
iloveyou
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 22ms/step
ilov

1/1 [==============================] - 0s 20ms/step
thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 21ms/step
thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 22ms/step
thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 22ms/step
thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 22ms/step
thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 25ms/step
thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 23ms/step
thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 22ms/step
thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 22ms/step
thanks
<class 'media

1/1 [==============================] - 0s 22ms/step
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 22ms/step
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 21ms/step
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 23ms/step
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 21ms/step
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 20ms/step
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 24ms/step
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 24ms/step
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 21ms/step
hello
<class 'mediapipe.pyth